In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale

In [2]:
# df = pd.read_csv("donors.csv", low_memory=False).drop(columns="Unnamed: 0")
# df.to_pickle("donors.pkl")

In [3]:
df = pd.read_pickle("donors.pkl")

In [4]:
sample = df.iloc[:1000, :].copy()

# EDA

In [5]:
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

groupMailOffers = sample.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

# dtale.show(sample[mailOrderOffers])

for col in groupMailOffers:
    sample[col] = sample[col].fillna(0).astype(np.int16)

In [6]:
searchName = "MDMAUD"
namesToGroup = [col for col in sample.columns.to_list() if searchName in col]
print(namesToGroup)

['MDMAUD', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A']


In [7]:
for name in namesToGroup:
    setattr(sample, name, sample[name].str.replace("X", "").replace("", np.nan))

In [8]:
# MAJOR
namesToGroup = ["MAJOR"] + namesToGroup
sample.MAJOR.replace({"X": True, " ": False}, inplace=True)

In [9]:
# RFA_2
groupRFA_2 = ["RFA_2", "RFA_2R", "RFA_2F", "RFA_2A"]
namesToGroup += groupRFA_2
sample[namesToGroup]

,MAJOR,MDMAUD,MDMAUD_R,MDMAUD_F,MDMAUD_A,RFA_2,RFA_2R,RFA_2F,RFA_2A
0,False,NaN,NaN,NaN,NaN,L4E,L,4,E
1,False,NaN,NaN,NaN,NaN,L2G,L,2,G
2,False,NaN,NaN,NaN,NaN,L4E,L,4,E
3,False,NaN,NaN,NaN,NaN,L4E,L,4,E
4,False,NaN,NaN,NaN,NaN,L2F,L,2,F
...,...,...,...,...,...,...,...,...,...
995,False,NaN,NaN,NaN,NaN,L2F,L,2,F
996,False,NaN,NaN,NaN,NaN,L1G,L,1,G
997,False,NaN,NaN,NaN,NaN,L1F,L,1,F
998,False,NaN,NaN,NaN,NaN,L4D,L,4,D


In [10]:
for c in namesToGroup:
    sample[c] = pd.Categorical(sample[c])

In [11]:
sample[namesToGroup].dtypes

MAJOR       category
MDMAUD      category
MDMAUD_R    category
MDMAUD_F    category
MDMAUD_A    category
RFA_2       category
RFA_2R      category
RFA_2F      category
RFA_2A      category
dtype: object

In [12]:
toReorder = ["ADATE", "RFA", "RDATE", "RAMNT"]
reorderedCols = []
for i in range(3, 25):
    reorderedCols += [str(col)+"_"+str(i) for col in toReorder]
reorderedCols = ["ADATE_2"] + groupRFA_2 + reorderedCols

dictIdx = {col: sample.columns.get_loc(col) for col in reorderedCols}

temp = sample[dictIdx.keys()].copy()
sample.drop(columns=dictIdx.keys(), inplace=True)

for key in dictIdx.keys():
    sample[key] = temp[key]

In [13]:
sample.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,RDATE_22,RAMNT_22,ADATE_23,RFA_23,RDATE_23,RAMNT_23,ADATE_24,RFA_24,RDATE_24,RAMNT_24
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,NaN,NaN,2014-07-01,S4E,2014-08-01,11.0,2014-06-01,S4E,2014-06-01,9.0
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,NaN,NaN,NaN,,NaN,NaN,2014-06-01,F1E,NaN,NaN
2,2010-01-01,AMH,1,NC,27017,,,NaN,0,,...,2014-09-01,8.0,2014-07-01,S4D,2014-07-01,7.0,2014-06-01,S3D,2014-06-01,6.0
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,NaN,NaN,2014-07-01,A3D,NaN,NaN,2014-06-01,A3D,NaN,NaN


In [16]:
sample.GEOCODE

0        
1      02
2        
3        
4        
       ..
995      
996      
997      
998      
999      
Name: GEOCODE, Length: 1000, dtype: object